In [ ]:
"Uni Face mask model"

In [38]:
#some important packages
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import model_from_json
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import argparse
import cv2
import os

In [39]:
# Load your trained  model

json_file = open('model_3class.json', 'r')
loaded_model_json = json_file.read()
json_file.close()# load json and create model

model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("model_3class.h5")
print("Loaded model from disk")

Loaded model from disk


In [40]:
# load our serialized face detector model from disk
print("[INFO] loading face detector model...")
prototxtPath = 'weights/deploy.prototxt.txt'
weightsPath = 'weights/res10_300x300_ssd_iter_140000.caffemodel'
net = cv2.dnn.readNet(prototxtPath, weightsPath)


[INFO] loading face detector model...


In [43]:
"" "Application on image // Play with the parameters to choose the right probabilities"""


'Application on image '

In [44]:
# load the image to test
#image = cv2.imread('images/pic1.jpeg')
image = cv2.imread('inputs/images/img4.jpeg')
orig = image.copy()
(h, w) = image.shape[:2]

# construct a blob from the image
blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300),
	(104.0, 177.0, 123.0))

# pass the blob through the network and obtain the face detections
print("[INFO] computing face detections...")
net.setInput(blob)
detections = net.forward()

# loop over the detections
for i in range(0, detections.shape[2]):
	# extract the confidence (i.e., probability) associated with
	# the detection
	confidence = detections[0, 0, i, 2]

	# filter out weak detections by ensuring the confidence is
	# greater than the minimum confidence
	if confidence > 0.5:
		# compute the (x, y)-coordinates of the bounding box for
		# the object
		box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
		(startX, startY, endX, endY) = box.astype("int")

		# ensure the bounding boxes fall within the dimensions of
		# the frame
		(startX, startY) = (max(0, startX), max(0, startY))
		(endX, endY) = (min(w - 1, endX), min(h - 1, endY))

		# extract the face ROI, convert it from BGR to RGB channel
		# ordering, resize it to 224x224, and preprocess it
		face = image[startY:endY, startX:endX]
		face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
		face = cv2.resize(face, (224, 224))
		face = img_to_array(face)
		face = preprocess_input(face)
		face = np.expand_dims(face, axis=0)

		# pass the face through the model to determine if the face
		# has a mask or not
		(notCorrect, mask, withoutMask) = model.predict(face)[0]

		# determine the class label and color we'll use to draw
		# the bounding box and text
		if (mask < 0.70 and withoutMask < 0.70) or notCorrect > max(mask, withoutMask):
			 label, color = "Not correct", (0, 255, 255)
		elif mask > max (withoutMask, notCorrect):
			label, color = "Mask", (0, 255,0)
		elif withoutMask > max (mask, notCorrect):
			label, color = "No Mask", (0, 0, 255)
		# label = "Mask" if mask > max (withoutMask, notCorrect) else "No Mask"
		# color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

		# include the probability in the label
		label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

		# display the label and bounding box rectangle on the output
		# frame
		cv2.putText(image, label, (startX, startY - 10),
			cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
		cv2.rectangle(image, (startX, startY), (endX, endY), color, 2)

#save the output image
cv2.imwrite('outputs/output_uni.jpg', image)
# show the output image
cv2.imshow("Output", image)
cv2.waitKey(0)

cv2.destroyAllWindows()

[INFO] computing face detections...


In [34]:
"""" Application on webcam """"

In [41]:
def detect_and_predict_mask(frame, faceNet, maskNet):
	# grab the dimensions of the frame and then construct a blob
	# from it
	(h, w) = frame.shape[:2]
	blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300),
		(104.0, 177.0, 123.0))

	# pass the blob through the network and obtain the face detections
	faceNet.setInput(blob)
	detections = faceNet.forward()

	# initialize our list of faces, their corresponding locations,
	# and the list of predictions from our face mask network
	faces = []
	locs = []
	preds = []

	# loop over the detections
	for i in range(0, detections.shape[2]):
		# extract the confidence (i.e., probability) associated with
		# the detection
		confidence = detections[0, 0, i, 2]

		# filter out weak detections by ensuring the confidence is
		# greater than the minimum confidence
		if confidence > 0.5:
			# compute the (x, y)-coordinates of the bounding box for
			# the object
			box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
			(startX, startY, endX, endY) = box.astype("int")

			# ensure the bounding boxes fall within the dimensions of
			# the frame
			(startX, startY) = (max(0, startX), max(0, startY))
			(endX, endY) = (min(w - 1, endX), min(h - 1, endY))

			# extract the face ROI, convert it from BGR to RGB channel
			# ordering, resize it to 224x224, and preprocess it
			face = frame[startY:endY, startX:endX]
			face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
			face = cv2.resize(face, (224, 224))
			face = img_to_array(face)
			face = preprocess_input(face)
			face = np.expand_dims(face, axis=0)

			# add the face and bounding boxes to their respective
			# lists
			faces.append(face)
			locs.append((startX, startY, endX, endY))

	# only make a predictions if at least one face was detected
	if len(faces) > 0:
		# for faster inference we'll make batch predictions on *all*
		# faces at the same time rather than one-by-one predictions
		# in the above `for` loop
		preds = maskNet.predict(faces)

	# return a 2-tuple of the face locations and their corresponding
	# locations
	return (locs, preds)



In [42]:
maskNet = model

size = 4
webcam = cv2.VideoCapture(0) #Use camera 0


# loop over the frames from the video stream
while True:
	# grab the frame from the threaded video stream and resize it
	# to have a maximum width of 400 pixels
	#frame = vs.read()
	#frame = imutils.resize(frame, width=400)

	(rval, frame) = webcam.read()
	#frame = imutils.resize(frame, width=400)
	frame = cv2.flip(frame,1,1) #Flip to act as a mirror
	frame = cv2.resize(frame, (frame.shape[1] // size, frame.shape[0] // size))# Resize the image to speed up detection
# detect faces in the frame and determine if they are wearing a
	# face mask or not
	(locs, preds) = detect_and_predict_mask(frame, net, maskNet)

	# loop over the detected face locations and their corresponding
	# locations
	for (box, pred) in zip(locs, preds):
		# unpack the bounding box and predictions
		(startX, startY, endX, endY) = box
		(notCorrect, mask, withoutMask) = pred

		# determine the class label and color we'll use to draw
		# the bounding box and text
		if (mask < 0.70 and withoutMask < 0.70) or notCorrect > max(mask, withoutMask):
			 label, color = "Not correct", (0, 255, 255)
		elif mask > max (withoutMask, notCorrect):
			label, color = "Mask", (0, 255,0)
		elif withoutMask > max (mask, notCorrect):
			label, color = "No Mask", (0, 0, 255)
		# include the probability in the label
		label = "{}: {:.2f}%".format(label, max(notCorrect, mask, withoutMask) * 100)

		# display the label and bounding box rectangle on the output
		# frame
		cv2.putText(frame, label, (startX-30, startY - 10),
			cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
		cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

	# show the output frame
	cv2.namedWindow("LIVE", cv2.WND_PROP_FULLSCREEN)
	cv2.setWindowProperty("LIVE",cv2.WND_PROP_FULLSCREEN,			cv2.WINDOW_FULLSCREEN)
	cv2.imshow("LIVE", frame)
	key = cv2.waitKey(10)
	if key==27:
		break
# Stop video
webcam.release()

# Close all started windows
cv2.destroyAllWindows()


In [ ]:
# Video